# C Interpreter

# Second Stage

Design a grammar using an aupward translator that accepts the following inputs from the C language:
- Conditional sentences (if - else)
- *printf* funcion

## Grammar design


### C Conditional statements (if - else)

In *C* the base structure of the conditional statement is:

``` 
    if(condition)
        instruction ;
```

Or could be:

```
    if(condition) {
        instruction ;
        instruction ;
        ...
        instruction ;
    }
```

The condition in *C* could be any expression.


We can represent this with the following grammar:

- exprIF -> if `(` condition `)` instruction `;`
- condition -> expr
- instruction -> `{` exprins `;` `}`
- instruction -> expr
- exprins -> exprins `;` expr
- exprins -> expr


Now we can include this into our grammar and test it, remember that
the condition and the instructions of the *if* statement has more priority so: 

- exprIF -> if `(` condition `)` instruction
- condition -> expr
- instruction -> `{` exprins `;` `}`
- instruction -> def
- exprins -> exprins `;` asing
- exprins -> asign


- def -> asign `;`
- asign -> ID `=` asign
- asign -> expr
- expr -> exprOR
- exprOR -> exprOR `||` exprAND
- exprOR -> exprAND
- exprAND -> exprAND `&&` exprE
- exprAND -> exprE
- exprE -> exprE `[==, !=]` exprC
- exprE -> exprC
- exprC -> exprC `[<, <=, >, >=]` exprA
- exprC -> exprA
- exprA -> exprA `[+, -] ` add
- exprA -> add
- add -> add `[*, /]` fact
- add -> fact
- fact -> `-` fact
- fact -> num
- fact -> `ID`

## Implementing the lexical analyzer (Lexer)

We will reuse the main lexical analyzer implementation, adding the *if* logic necessary to process the new rules

In [2]:
from sly import Lexer

class Scanner(Lexer):
    tokens = {ID, CNUM, EQ, NEQ, OR, AND, LEQ, GEQ, IF}
    literals ={'=', '<', '>', '/', '!', '+', '-' , '*', ';', '{', '}', '(', ')'}

    # Ignore whitespace and tabulations

    ignore = ' \t'

    # Regular expressions rules for tokens

    ID = r'[a-zA-Z][\w_]*'
    EQ = r'=='
    NEQ = r'!='
    GEQ = r'>='
    LEQ = r'<='
    AND = r'&&'
    OR = r'\|\|'

    # Special cases
    ID['if'] = IF

    @_(r'\d+')
    def CNUM(self, t):
        t.value = int(t.value)
        return t

    # Error handling rule

    def error(self, t):
        print('<-'*10,"Illegal character '{}'".format(t.value[0]), '->'*10)
        self.index += 1
        t.type='Illegal'
        t.value =t.value[0]
        return t

## Testing the lexical analyzer (Lexer)


### Importing data

In [66]:
import pandas as pd 

data = pd.read_csv('../assets/testing/if_sentences.csv')
data[['if_sentences']]

,if_sentences
0,if(a > 5) { b = 4; c = 5 + b; }
1,if(b < a) c = 4;
2,if(3 > 5) { b = 4; c = 5 + b; d = 5 + 4 ;}


In [67]:
lexer = Scanner()
sentences = data['if_sentences'].values
pass_or_not = []
all_token_pass = True

for index, sentence in enumerate(sentences):
    print('-' * 80,"{} Lexically Testing sentence: '{}'".format(index, sentence),'-' * 80, sep='\n')
    for token in lexer.tokenize(sentence):
        print(" type = '{}', value = '{}'".format(token.type, token.value))
        if all_token_pass and 'Illegal' in token.type:
            all_token_pass = False
    
    pass_or_not.append('Pass') if all_token_pass else pass_or_not.append('FAIL')
    all_token_pass = True

data['Test'] = pass_or_not

--------------------------------------------------------------------------------
0 Lexically Testing sentence: 'if(a > 5) { b = 4; c = 5 + b; }'
--------------------------------------------------------------------------------
 type = 'IF', value = 'if'
 type = '(', value = '('
 type = 'ID', value = 'a'
 type = '>', value = '>'
 type = 'CNUM', value = '5'
 type = ')', value = ')'
 type = '{', value = '{'
 type = 'ID', value = 'b'
 type = '=', value = '='
 type = 'CNUM', value = '4'
 type = ';', value = ';'
 type = 'ID', value = 'c'
 type = '=', value = '='
 type = 'CNUM', value = '5'
 type = '+', value = '+'
 type = 'ID', value = 'b'
 type = ';', value = ';'
 type = '}', value = '}'
--------------------------------------------------------------------------------
1 Lexically Testing sentence: 'if(b < a) c = 4; '
--------------------------------------------------------------------------------
 type = 'IF', value = 'if'
 type = '(', value = '('
 type = 'ID', value = 'b'
 type = '<', value 

In [5]:
data

,if_sentences,Test
0,if(a > 5) { b = 4; c = 5 + b; },Pass
1,if(b < a) c = 4;,Pass


## Implementing the Grammar and Semantic Analyzer (Parser)

In [158]:
from sly import Parser

class CInterpreterParser(Parser):
    tokens = Scanner.tokens

    def __init__(self):
        self.table = {}
    
    # - exprIF -> if `(` condition `)` instruction
    # - condition -> expr
    # - instruction -> `{` exprins; `}`
    # - instruction -> def
    # - exprins -> exprins `;` asign 
    # - exprins -> asign

    # - def -> asign `;`
    # - asign -> ID `=` asign
    # - asign -> expr
    # - expr -> exprOR
    # - exprOR -> exprOR `||` exprAND
    # - exprOR -> exprAND
    # - exprAND -> exprAND `&&` exprE
    # - exprAND -> exprE
    # - exprE -> exprE `[==, !=]` exprC
    # - exprE -> exprC
    # - exprC -> exprC `[<, <=, >, >=]` exprA
    # - exprC -> exprA
    # - exprA -> exprA `[+, -] ` add
    # - exprA -> add
    # - add -> add `[*, /]` fact
    # - add -> fact
    # - fact -> `-` fact
    # - fact -> num
    # - fact -> `ID`

    @_('IF "(" condition ")" instruction')
    def exprIF(self, p):
        print('if(', p.condition, ')')
        if p.condition:
            print(p.instruction)

    @_('expr')
    def condition(self, p):
        return p.expr


    @_('"{" exprins ";" "}"')
    def instruction(self, p):
        return p.exprins + ' ;'

    @_('def_')
    def instruction(self, p):
        return p.def_

    @_('exprins ";" asign')
    def exprins(self, p):
        return str(p.exprins) +  " ; " + str(p.asign)

    @_('asign')
    def exprins(self, p):
        return p.asign

    @_('asign ";"')
    def def_(self, p):
        return str(p.asign) + ' ;'

    @_('ID "=" asign')
    def asign(self, p):
        self.table[p.ID] = p.asign  
        return p.ID + ' = ' + str(p.asign)

    @_('expr')
    def asign(self, p):
        return p.expr

    @_('exprOR')
    def expr(self, p):
        return p.exprOR

    @_('exprOR OR exprAND')
    def exprOR(self, p):
        return bool(p.exprOR) | bool(p.exprAND)

    @_('exprAND')
    def exprOR(self, p):
        return p.exprAND

    @_('exprAND AND exprE')
    def exprAND(self, p):
        return bool(p.exprAND) & bool(p.exprE)

    @_('exprE') 
    def exprAND(self, p):
        return p.exprE

    @_('exprE NEQ exprC')
    def exprE(self, p):
        return p.exprE != p.exprC

    @_('exprE EQ exprC')
    def exprE(self, p):
        return p.exprE == p.exprC

    @_('exprC')
    def exprE(self, p):
        print(p.exprC)
        return p.exprC
    
    @_('exprC GEQ exprA')
    def exprC(self, p):
        return p.exprC < p.exprA

    @_('exprC ">" exprA')
    def exprC(self, p):
        return p.exprC > p.exprA

    @_('exprC LEQ exprA')
    def exprC(self, p):
        return p.exprC <= p.exprA

    @_('exprC "<" exprA')
    def exprC(self, p):
        return p.exprC <p.exprA

    @_('exprA')
    def exprC(self, p):
        return p.exprA

    @_('exprA "-" add')
    def exprA(self, p):
        return p.exprA - p.add
    
    @_('exprA "+" add')
    def exprA(self, p):
        return p.exprA + p.add

    @_('add')
    def exprA(self, p):
        return p.add

    @_('add "/" fact')
    def add(self, p):
        return p.add / p.fact

    @_('add "*" fact')
    def add(self, p):
        return p.add * p.fact

    @_('fact')
    def add(self, p):
        return p.fact

    @_('"-" fact')
    def fact(self, p):
        return -p.fact

    @_('CNUM')
    def fact(self, p):
        return p.CNUM

    @_('ID')
    def fact(self, p):
        try:
            return self.table[p.ID]
        except LookupError:
            print("Undefined name {}".format(p.ID))
            return 0


In [160]:
if __name__ == '__main__':
    lexer = Scanner()
    parser = CInterpreterParser()
    string = "if(2 > 5) { b = 4; c = 3 + b; p = 3; }"
    parser.parse(lexer.tokenize(string))

False
4
7
3
if( False )


In [161]:
parser.table

{'b': 4, 'c': 7, 'p': 3}